In [3]:
import duckdb

con = duckdb.connect()
con.execute("LOAD httpfs;")
con.execute("LOAD aws;")
con.execute("""
    SET s3_endpoint='minio:9000';
    SET s3_access_key_id='datalake';
    SET s3_secret_access_key='datalake';
    SET s3_use_ssl=false;
    SET s3_url_style='path';
""")
try:
    print("Searching for buckets and files...")
    # This glob will search for any parquet files in any bucket
    res = con.execute("SELECT * FROM glob('s3://trusted/**/*.parquet')").df()
    if res.empty:
        print("Connected, but no .parquet files found. Check your bucket names.")
    else:
        display(res)
except Exception as e:
    print(f"Still failing: {e}")
    print(f"Connection Failed: {e}")

Searching for buckets and files...


,file
0,s3://trusted/gtfs/calendar/calendar.parquet
1,s3://trusted/gtfs/frequencies/frequencies.parquet
2,s3://trusted/gtfs/routes/routes.parquet
3,s3://trusted/gtfs/stop_times/stop_times.parquet
4,s3://trusted/gtfs/stops/stops.parquet
...,...
989,s3://trusted/sptrans/routes/routes.parquet
990,s3://trusted/sptrans/stop_times/stop_times.par...
991,s3://trusted/sptrans/stops/stops.parquet
992,s3://trusted/sptrans/trip_details/trip_details...


In [4]:
# Limit DuckDB to 4GB (or whatever suits your machine)
con.execute("SET memory_limit='4GB';")

In [18]:
def query_positions_for_line_vehicle__sentido_day(linha_lt, veiculo_id, linha_sentido, day):
    bucket_name = "trusted"
    app_folder = "sptrans"
    positions_table_name = "positions"
    s3_path = f"s3://{bucket_name}/{app_folder}/{positions_table_name}/**"
    sql = f"""
        SELECT 
            veiculo_ts, linha_lt, veiculo_id, linha_sentido, 
            distance_to_first_stop, distance_to_last_stop, 
            is_circular, lt_origem, lt_destino
        FROM read_parquet('{s3_path}', hive_partitioning = true)
        WHERE linha_lt = '{linha_lt}' and veiculo_id = '{veiculo_id}' and day = '{day}'
        AND linha_sentido = '1'
        ORDER BY veiculo_ts ASC;
    """
    df = con.execute(sql).df()
    
    display(df.shape)
    import pandas as pd
    
    # This forces the display of all 100 rows and all columns
    with pd.option_context('display.max_rows', 200, 'display.max_columns', None):
        display(df.head(200))

In [15]:
def query_positions_for_line_vehicle_day(linha_lt, veiculo_id, day):
    bucket_name = "trusted"
    app_folder = "sptrans"
    positions_table_name = "positions"
    s3_path = f"s3://{bucket_name}/{app_folder}/{positions_table_name}/**"
    sql = f"""
        SELECT 
            veiculo_ts, linha_lt, veiculo_id, linha_sentido, 
            distance_to_first_stop, distance_to_last_stop, 
            is_circular, lt_origem, lt_destino
        FROM read_parquet('{s3_path}', hive_partitioning = true)
        WHERE linha_lt = '{linha_lt}' and veiculo_id = '{veiculo_id}' and day = '{day}'
        ORDER BY veiculo_ts ASC;
    """
    df = con.execute(sql).df()
    
    display(df.shape)
    import pandas as pd
    
    # This forces the display of all 100 rows and all columns
    with pd.option_context('display.max_rows', 200, 'display.max_columns', None):
        display(df.head(200))

In [19]:
linha_lt = '8003-10'
veiculo_id = 10657
linha_sentido = '1'
day = '10'

query_positions_for_line_vehicle__sentido_day(linha_lt, veiculo_id, linha_sentido, day)

(54, 9)

,veiculo_ts,linha_lt,veiculo_id,linha_sentido,distance_to_first_stop,distance_to_last_stop,is_circular,lt_origem,lt_destino
0,2026-02-10 00:00:25+00:00,8003-10,10657,1,38.0,5875.0,False,REMÉDIOS,TERM. LAPA
1,2026-02-10 00:02:06+00:00,8003-10,10657,1,552.0,5552.0,False,REMÉDIOS,TERM. LAPA
2,2026-02-10 00:04:33+00:00,8003-10,10657,1,875.0,5411.0,False,REMÉDIOS,TERM. LAPA
3,2026-02-10 00:06:02+00:00,8003-10,10657,1,1276.0,4960.0,False,REMÉDIOS,TERM. LAPA
4,2026-02-10 00:08:06+00:00,8003-10,10657,1,1966.0,4212.0,False,REMÉDIOS,TERM. LAPA
5,2026-02-10 00:10:19+00:00,8003-10,10657,1,2386.0,3553.0,False,REMÉDIOS,TERM. LAPA
6,2026-02-10 00:12:32+00:00,8003-10,10657,1,3301.0,2595.0,False,REMÉDIOS,TERM. LAPA
7,2026-02-10 00:14:01+00:00,8003-10,10657,1,3669.0,2259.0,False,REMÉDIOS,TERM. LAPA
8,2026-02-10 00:16:30+00:00,8003-10,10657,1,4336.0,1774.0,False,REMÉDIOS,TERM. LAPA
9,2026-02-10 00:18:19+00:00,8003-10,10657,1,5275.0,889.0,False,REMÉDIOS,TERM. LAPA


In [20]:
linha_lt = '8003-10'
veiculo_id = 10657
day = '10'

query_positions_for_line_vehicle_day(linha_lt, veiculo_id, day)

(121, 9)

,veiculo_ts,linha_lt,veiculo_id,linha_sentido,distance_to_first_stop,distance_to_last_stop,is_circular,lt_origem,lt_destino
0,2026-02-10 00:00:25+00:00,8003-10,10657,1,38.0,5875.0,False,REMÉDIOS,TERM. LAPA
1,2026-02-10 00:02:06+00:00,8003-10,10657,1,552.0,5552.0,False,REMÉDIOS,TERM. LAPA
2,2026-02-10 00:04:33+00:00,8003-10,10657,1,875.0,5411.0,False,REMÉDIOS,TERM. LAPA
3,2026-02-10 00:06:02+00:00,8003-10,10657,1,1276.0,4960.0,False,REMÉDIOS,TERM. LAPA
4,2026-02-10 00:08:06+00:00,8003-10,10657,1,1966.0,4212.0,False,REMÉDIOS,TERM. LAPA
5,2026-02-10 00:10:19+00:00,8003-10,10657,1,2386.0,3553.0,False,REMÉDIOS,TERM. LAPA
6,2026-02-10 00:12:32+00:00,8003-10,10657,1,3301.0,2595.0,False,REMÉDIOS,TERM. LAPA
7,2026-02-10 00:14:01+00:00,8003-10,10657,1,3669.0,2259.0,False,REMÉDIOS,TERM. LAPA
8,2026-02-10 00:16:30+00:00,8003-10,10657,1,4336.0,1774.0,False,REMÉDIOS,TERM. LAPA
9,2026-02-10 00:18:19+00:00,8003-10,10657,1,5275.0,889.0,False,REMÉDIOS,TERM. LAPA
